In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pymongo import MongoClient
import tensorflow as tf
from tensorflow import keras

In [18]:
# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/Scibio")  # Remplacez par l'URL de votre serveur MongoDB
db = client["Scribio"]  # Nom de la base de données
collection = db["Dataset"]  # Nom de la collection
# Temps : 0.0s

In [ ]:
# Récupérer le df d'entrainement depuis MongoDB
document_recupere = collection.find({"Repartition": "train"}) # Récupérer les documents de la collection 'train'
df_train_mongoDB = pd.DataFrame(list(document_recupere)) # Convertir le curseur en DataFrame
df_train_mongoDB = df_train_mongoDB.drop(columns=["_id"]) # Supprimer la colonne _id
df_train = df_train_mongoDB.sample(frac=1).reset_index(drop=True) # Mélanger les données + Reindexer
print(df_train.shape) # ()
df_train.head()
# Temps : 

In [21]:
# Récupérer le df de test depuis MongoDB
document_recupere = collection.find({"Repartition": "test"}) # Récupérer les documents de la collection 'test'
df_test_mongoDB = pd.DataFrame(list(document_recupere)) # Convertir le curseur en DataFrame
df_test_mongoDB = df_test_mongoDB.drop(columns=["_id"]) # Supprimer la colonne _id
df_test = df_test_mongoDB.sample(frac=1).reset_index(drop=True) # Mélanger les données + Reindexer
print(df_test.shape) # ()
df_test.head()
# Temps : 

<class 'list'>
(3, 5)


,_id,Label,Repartition,Chemin,Tableau_Pixels
0,677fde52eaf2499095e8df9a,A,test,../Dataset/dataset/test/A/001452.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,677fde54eaf2499095e8df9b,B,test,../Dataset/dataset/test/B/000876.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,677fde67eaf2499095e8df9c,C,test,../Dataset/dataset/test/C/002302.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [7]:
# Transformation des colonnes des df en tableau numpy
X_train = df_train['Tableau_Pixels']
y_train = df_train['Label']
X_test = df_test['Tableau_Pixels']
y_test = df_test['Label']
# Temps : 

In [ ]:
# Reseau de neurones
model = keras.Sequential([
    keras.layers(input_shape=(2500,)), # input layer
    keras.layers.Dense(700, activation='relu'), # hidden layer
    keras.layers.Dense(600, activation='relu'), # hidden layer
    keras.layers.Dense(500, activation='relu'), # hidden layer
    keras.layers.Dense(400, activation='relu'), # hidden layer
    keras.layers.Dense(300, activation='relu'), # hidden layer
    keras.layers.Dense(200, activation='relu'), # hidden layer
    keras.layers.Dense(100, activation='relu'), # hidden layer
    keras.layers.Dense(50, activation='relu'), # hidden layer
    keras.layers.Dense(10, activation='sigmoid') # output layer
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
# Sauvegarder le modèle
model.save("../Scribio.h5")  # Format HDF5 (.h5)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted[1]

In [ ]:
np.argmax(y_predicted[1])

In [ ]:
y_predicted_labels = [np.argmax(i) for i in y_predicted]
y_predicted_labels[:5]

In [ ]:
# Matrice de confusion
cm = tf.math.confusion_matrix(labels=y_test, predictions=y_predicted_labels)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')